In [1]:
%cd ..
import os, sys
sys.path.append(os.path.join(os.getcwd(), "src"))

d:\Blockchain\data\orai\orchai_validator


In [2]:
from orchai.upload import ETLProcessor, get_spark, sampling, uploading
from omegaconf import OmegaConf
import pyspark.sql.functions as F

In [3]:
config = OmegaConf.load("config/local_file.yaml")
spark = get_spark()
sb = 7059473
eb = 7071323
print((eb - sb) / 150 + 1)
df = sampling(spark, config, sb, eb)
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

80.0
------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [4]:
dff.printSchema()

root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,6) (nullable = true)
 |-- delegators_token: decimal(38,6) (nullable = true)
 |-- voting_power_score: decimal(38,6) (nullable = true)
 |-- commission_score: double (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- vote_score: double (nullable = true)
 |-- score: double (nullable = true)
 |-- label: double (nullable = true)



In [5]:
dff.count()

2691

In [6]:
dff.groupBy("block_height").count().count()

69

In [10]:
import numpy as np

bh = dff.groupBy("block_height").count().select(F.col("block_height"))
bb = bh.toPandas()
bh = bb.to_numpy().reshape((-1,))
bh.sort()
print(bh[0], bh[-1])
diff = np.diff(bh)
for i, d in enumerate(diff):
    if d != 150:
        print(bh[i], bh[i + 1])

7059923 7070123


In [11]:
7059473 - 7059923

-450

In [12]:
7071323 - 7070123

1200